### Preparação do Diretório

In [4]:
'''
Definição da função auxiliar
'''
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def ajuste(data_dir, target_size, batch_size):
    datagen = ImageDataGenerator(
        rescale=1.0 / 255,  # normalização
        validation_split=0.2  # separação de teste e treino
    )
    
    # definir porção de dados para treino
    train = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    
    # definir porção de dados para teste
    test = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train, test

In [5]:
'''
Aplicação da função auxiliar
'''

# caminho do diretório
data_dir = r'C:\Users\User\UnB\cis\Trainee\Semana4\animals'

# tamanho da imagem
TARGET_SIZE = (224, 224)

# tamanho do lote
BATCH_SIZE = 32

train, test = ajuste(data_dir, TARGET_SIZE, BATCH_SIZE)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


### Definição da Rede Convolucional

In [19]:
'''
Criação do CNN com 1 hidden layer
'''
import tensorflow as tf


def CNN():
    model = tf.keras.Sequential([
        
        # primeira camada (feature learning)
        tf.keras.layers.Conv2D(32, kernel_size=3, activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        
        # segunda camada (feature learning)
        tf.keras.layers.Conv2D(64, kernel_size=3, activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        
        # camada final (classification)
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation="relu"),
        tf.keras.layers.Dense(3, activation="softmax") # 3 outputs de classificação distintos
    ])
    
    return model

### Avaliação do modelo

In [20]:
# criar modelo
modelo = CNN()

# compilar modelo
modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# treinar modelo
epochs = 10
modelo.fit(train, epochs=epochs, validation_data=test)

# Avaliar o modelo
evaluation = modelo.evaluate(test)
print("Loss:", evaluation[0])
print("Accuracy:", evaluation[1])

# Fazer previsões em novos dados
new_data = test.next()
predictions = modelo.predict(new_data[0])


Epoch 1/10
75/75 [==============================] - 204s 3s/step - loss: 2.7106 - accuracy: 0.5625 - val_loss: 0.7333 - val_accuracy: 0.6417
Epoch 2/10
75/75 [==============================] - 202s 3s/step - loss: 0.5770 - accuracy: 0.7321 - val_loss: 0.7413 - val_accuracy: 0.6633
Epoch 3/10
75/75 [==============================] - 210s 3s/step - loss: 0.3791 - accuracy: 0.8521 - val_loss: 0.8041 - val_accuracy: 0.6783
Epoch 4/10
75/75 [==============================] - 211s 3s/step - loss: 0.1811 - accuracy: 0.9337 - val_loss: 0.8601 - val_accuracy: 0.6683
Epoch 5/10
75/75 [==============================] - 223s 3s/step - loss: 0.0871 - accuracy: 0.9733 - val_loss: 1.2923 - val_accuracy: 0.6800
Epoch 6/10
75/75 [==============================] - 253s 3s/step - loss: 0.0769 - accuracy: 0.9800 - val_loss: 1.3560 - val_accuracy: 0.6800
Epoch 7/10
75/75 [==============================] - 251s 3s/step - loss: 0.0322 - accuracy: 0.9908 - val_loss: 1.7531 - val_accuracy: 0.6483
Epoch 8/10
75

### Transfer Learning

In [23]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16


# Carregar modelo VGG16 pré-treinado sem camadas de classificação
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adicionar camadas personalizadas (add batch normalization)
modelo_transfer = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu')
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compilar o modelo
modelo_transfer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
epochs = 10
modelo_transfer.fit(train, epochs=epochs, validation_data=test)

# Avaliar o modelo
evaluation_transfer = modelo_transfer.evaluate(test)
print("Transfer Learning - Loss:", evaluation_transfer[0])
print("Transfer Learning - Accuracy:", evaluation_transfer[1])


Epoch 1/10
75/75 [==============================] - 2240s 30s/step - loss: 1.2143 - accuracy: 0.3079 - val_loss: 1.0988 - val_accuracy: 0.3333
Epoch 2/10
75/75 [==============================] - 1962s 26s/step - loss: 1.0992 - accuracy: 0.3446 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 3/10
75/75 [==============================] - 2101s 28s/step - loss: 1.0992 - accuracy: 0.3258 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 4/10
75/75 [==============================] - 1945s 26s/step - loss: 1.0989 - accuracy: 0.3367 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 5/10
75/75 [==============================] - 1944s 26s/step - loss: 1.0992 - accuracy: 0.3221 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 6/10
75/75 [==============================] - 1931s 26s/step - loss: 1.0987 - accuracy: 0.3217 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 7/10
75/75 [==============================] - 1926s 26s/step - loss: 1.0987 - accuracy: 0.3308 - val_loss: 1.0986 - val_accuracy: 0.3333

In [6]:
'''
Considerando a piora do modelo com o uso do transfer learning, fez-se uma nova tentativa, alterando-se alguns parâmetros para
verificar a possibilidade de melhoria. Essa nova tentetiva é percebida abaixo, após os feedbacks dados na semanal
'''
import tensorflow as tf
from tensorflow.keras.applications import VGG16


# Carregar modelo VGG16 pré-treinado sem camadas de classificação
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adicionar camadas personalizadas (add batch normalization)
modelo_transfer = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(32),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compilar o modelo
modelo_transfer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
epochs = 10
modelo_transfer.fit(train, epochs=epochs, validation_data=test)

# Avaliar o modelo
evaluation_transfer = modelo_transfer.evaluate(test)
print("Transfer Learning - Loss:", evaluation_transfer[0])
print("Transfer Learning - Accuracy:", evaluation_transfer[1])

Epoch 1/10
75/75 [==============================] - 1957s 26s/step - loss: 0.9846 - accuracy: 0.5150 - val_loss: 0.8121 - val_accuracy: 0.5800
Epoch 2/10
75/75 [==============================] - 2027s 27s/step - loss: 0.8759 - accuracy: 0.5683 - val_loss: 1.6702 - val_accuracy: 0.3700
Epoch 3/10
75/75 [==============================] - 1981s 26s/step - loss: 0.8610 - accuracy: 0.5633 - val_loss: 1.0755 - val_accuracy: 0.4633
Epoch 4/10
75/75 [==============================] - 1924s 26s/step - loss: 0.7834 - accuracy: 0.5754 - val_loss: 2.7812 - val_accuracy: 0.3483
Epoch 5/10
75/75 [==============================] - 1922s 26s/step - loss: 0.7693 - accuracy: 0.5983 - val_loss: 1.3228 - val_accuracy: 0.3350
Epoch 6/10
75/75 [==============================] - 1942s 26s/step - loss: 0.7138 - accuracy: 0.6137 - val_loss: 1.2410 - val_accuracy: 0.4183
Epoch 7/10
75/75 [==============================] - 1971s 26s/step - loss: 0.7013 - accuracy: 0.6196 - val_loss: 0.8455 - val_accuracy: 0.6117

### Implementação da Ideia de Regularização

In [24]:
'''
Inserção de camadas de dropout para reduzir overfitting
'''
import tensorflow as tf


def reg_CNN():
    model = tf.keras.Sequential([
        
        # primeira camada (feature learning)
        tf.keras.layers.Conv2D(32, kernel_size=3, activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(0.25), # aplicação da ideia de regularização para reduzir overfitting
        
        # segunda camada (feature learning)
        tf.keras.layers.Conv2D(64, kernel_size=3, activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(0.25), # aplicação da ideia de regularização para reduzir overfitting
        
        # camada final (classification)
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(3, activation="softmax") # 3 outputs de classificação distintos
    ])
    
    return model

In [25]:
# criar modelo
modelo1 = reg_CNN()

# compilar modelo
modelo1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# treinar modelo
epochs = 10
modelo1.fit(train, epochs=epochs, validation_data=test)

# Avaliar o modelo
evaluation = modelo1.evaluate(test)
print("Loss:", evaluation[0])
print("Accuracy:", evaluation[1])

# Fazer previsões em novos dados
new_data = test.next()
predictions = modelo1.predict(new_data[0])

Epoch 1/10
75/75 [==============================] - 256s 3s/step - loss: 4.4601 - accuracy: 0.4675 - val_loss: 0.9047 - val_accuracy: 0.5533
Epoch 2/10
75/75 [==============================] - 243s 3s/step - loss: 0.8135 - accuracy: 0.6142 - val_loss: 0.9705 - val_accuracy: 0.4667
Epoch 3/10
75/75 [==============================] - 242s 3s/step - loss: 0.6825 - accuracy: 0.6708 - val_loss: 0.8649 - val_accuracy: 0.5250
Epoch 4/10
75/75 [==============================] - 243s 3s/step - loss: 0.5484 - accuracy: 0.7442 - val_loss: 0.8924 - val_accuracy: 0.5733
Epoch 5/10
75/75 [==============================] - 242s 3s/step - loss: 0.3923 - accuracy: 0.8183 - val_loss: 1.0638 - val_accuracy: 0.5733
Epoch 6/10
75/75 [==============================] - 243s 3s/step - loss: 0.2839 - accuracy: 0.8800 - val_loss: 1.2781 - val_accuracy: 0.5767
Epoch 7/10
75/75 [==============================] - 248s 3s/step - loss: 0.1944 - accuracy: 0.9208 - val_loss: 1.3865 - val_accuracy: 0.5567
Epoch 8/10
75